# Two Stages Factory Planning w/ Quality

This notebook presents the two stages factory planning algorithm for factorio. It handles (qualities)[https://factorio.com/blog/post/fff-375] and modules correctly. 


The first stage (macro) computes what how many building + recipe are required to create a fixed set of item flow. The second stages (micro) compute a flow-aware bin packing combinaison to link buildings together.

Both stages are described in more precision in the following section & code.

In [1]:
import gurobipy as grb
from gurobipy import GRB
import networkx as nx

### Stage 0: Data loading

The data loading creates a graph from the raw factorio data with informations for recipes, machines, and modules. The graph is bipartite, with on one side items/fluids, and on the other side recipes $\times$ machines $\times$ module. More precisely,


$$ G = (\mathtt{Items} \cup \mathtt{Machines}, E) $$

Where
- $\mathtt{Items}$ are items and fluid with an additional integer indicating the quality. The tagged union is build from Factorio's raw data. The tuples are of the form `("item", "iron-plate", 3)` where the first element is either `"item"` or `"fluid"`. The second it the item/fluid's name. And the third is either 0 for normal quality, 1 for uncommon, 2 for rare, 3 for epic and 4 for legendary.
- $\mathtt{Machines} = \mathtt{Recipe} \times \mathtt{AssemblyMachine} \times \mathtt{ModuleChoice} \times \mathtt{Quality}$ are recipes with their compatible assembly machines/furnaces. The module choice is a list of modules compatible with both the recipe and the machine. The quality is associated with the recipe and not the assembly machine.

Edges are split into ingredients and results for the recipes.

- Ingredients: $E \cap (\mathtt{Items} \times \mathtt{Machines})$.
- Results: $E \cap (\mathtt{Machines} \times \mathtt{Items})$.

Edges are annotated with expected value of item flow per instance of the machine in the `per_machine` attribute. This computed value is key in the further processings steps of stage 1 and stage 2.



##### Data loading

Data is loaded from the raw json generated by factorio iteself. To generate the data from other mods you need first to load the selected mods, then quit factorio and launch it from the command line with the `--dump-data` option:

```bash
factorio --dump-data 
```

The data will be available in the `script/data-raw-dump.json` file in your [application directory](https://wiki.factorio.com/Application_directory).




In [2]:
import json

space_age = json.load(open("data/space-age.json"))


In [3]:
G = nx.DiGraph()
from collections import defaultdict


machines = defaultdict(set)
for n, f in space_age["furnace"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("furnace", n))

for n, f in space_age["assembling-machine"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("assembling-machine", n))


modules = defaultdict(dict)
for n, m in space_age["module"].items():
    modules[m["category"]][m["name"]] = m["effect"]

print(modules)

max_quality_level = 0
min_quality_level = float("inf")
for n, q in space_age["quality"].items():
    max_quality_level = max(max_quality_level, q["level"])
    min_quality_level = min(min_quality_level, q["level"])

items = set()

for n, r in space_age["recipe"].items():
    if "ingredients" in r:
        items.update((i["type"], i["name"]) for i in r["ingredients"])
    if "results" in r:
        items.update((i["type"], i["name"]) for i in r["results"])


defaultdict(<class 'dict'>, {'speed': {'speed-module': {'speed': 0.2, 'consumption': 0.5, 'quality': -0.1}, 'speed-module-2': {'speed': 0.3, 'consumption': 0.6, 'quality': -0.15}, 'speed-module-3': {'speed': 0.5, 'consumption': 0.7, 'quality': -0.25}}, 'efficiency': {'efficiency-module': {'consumption': -0.3}, 'efficiency-module-2': {'consumption': -0.4}, 'efficiency-module-3': {'consumption': -0.5}}, 'productivity': {'productivity-module': {'productivity': 0.04, 'consumption': 0.4, 'pollution': 0.05, 'speed': -0.05}, 'productivity-module-2': {'productivity': 0.06, 'consumption': 0.6, 'pollution': 0.07, 'speed': -0.1}, 'productivity-module-3': {'productivity': 0.1, 'consumption': 0.8, 'pollution': 0.1, 'speed': -0.15}}, 'quality': {'quality-module': {'quality': 0.1, 'speed': -0.05}, 'quality-module-2': {'quality': 0.2, 'speed': -0.05}, 'quality-module-3': {'quality': 0.25, 'speed': -0.05}}})


This is the cell where the module choices can be tuned.

Having too much module choices puts a huge stain on the solver capabilities because there is too much variables. Moreover, one can expect that using only the best modules available to your factory is better. To modify the available modules, you can use the `possible_modules` variable. You need to filter amoung available modules ccording to the recipe and machine. 


Each machine + recipes are then filled with every possible combinaison of modules, creating additional nodes and edges to the graph. To get an order of magnitude, the graph with only quality and speed 3 modules has 48596 nodes for space age, if considering all tier 3 modules, we have 187483 nodes. When considering all possibilities for modules of all levels, the graphs fails to generate in reasonable time. (less than few minutes).

In [4]:

from itertools import chain, combinations_with_replacement

all_crafting_variables = set()
all_item_variables = set()

ingredients_to_recipes = defaultdict(set)
products_to_recipes = defaultdict(set)


def get_effect(chosen_modules):
    effects = {
        "consumption": 1.0,
        "speed": 1.0,
        "productivity": 0.0,
        "pollution": 1.0,
        "quality": 0.0,
    }

    for m in chosen_modules:
        module = space_age["module"][m]
        for k in effects:
            effects[k] += module["effect"].get(k, 0)
    
    effects["speed"] = max(effects["speed"], 0.2)
    effects["consumption"] = max(effects["consumption"], 0.2)
    effects["productivity"] = max(effects["productivity"], 0.0)
    effects["pollution"] = max(effects["pollution"], 0.2)
    effects["quality"] = max(effects["quality"], 0.)

    return effects

for _, r in space_age["recipe"].items():
    allow_quality = int(r.get("allow_quality", True))
    for q in range(allow_quality * (max_quality_level)+1):

        for i in r.get("ingredients", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))
        for i in r.get("results", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))


        for m in machines[r.get("category", "crafting")]:
            machine = space_age[m[0]][m[1]]

            if m == ('assembling-machine', 'biochamber'):
                continue

            slots = machine.get("module_slots", 0)
            possible_modules_categories = machine.get("allowed_module_categories", modules.keys())
            possible_modules = set().union(*(set(modules[c].keys()) for c in possible_modules_categories))

            # possible_modules = {m for m in possible_modules if space_age["module"][m].get("tier", 1) == 3}

            possible_modules = {m for m in possible_modules if m in {"quality-module-3", "speed-module-3"}}

        
            for chosen_modules in chain.from_iterable(combinations_with_replacement(possible_modules, i) for i in range(slots)):

                key = (r["name"], q, m, chosen_modules)
                all_crafting_variables.add(key)

                effects = get_effect(chosen_modules)

                for k, i in enumerate(r.get("ingredients", [])):
                    ingredients_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))
                for k, i in enumerate(r.get("results", [])):
                    if effects["quality"] > 0:
                        for qual in range(q, max_quality_level+1):
                            products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else qual)].add((key, k))
                    products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))


In [5]:
import networkx as nx

G = nx.DiGraph()

# --- Add nodes ---

for item_key in all_item_variables:
    itype, iname, iq = item_key
    G.add_node(item_key, kind="item", itype=itype, name=iname, quality=iq)

for machine_key in all_crafting_variables:
    rname, q, m, mods = machine_key
    G.add_node(machine_key, kind="recipe", recipe=rname, quality=q, machine=m, mods=mods)

# --- Add ingredient edges: item -> recipe ---

for item_key in all_item_variables:
    for recipe_key, k in ingredients_to_recipes.get(item_key, []):
        rname, q, m, mods = recipe_key
        r = space_age["recipe"][rname]
        ing = r.get("ingredients", [])[k]

        amt = ing.get("amount", 0)
        if not amt:
            continue

        # item consumed per craft of recipe_key
        G.add_edge(
            item_key,
            recipe_key,
            kind="ingredient",
            per_machine=amt,
        )

# --- Add product edges: recipe -> item ---

alpha = 0.1  # 10% chance to upgrade again

def quality_prob(base_q, target_q, Q):
    # No quality modules or targeting below base quality
    if Q <= 0.0 or target_q < base_q:
        return 1.0 if target_q == base_q else 0.0

    # unsure of this one.
    if Q > 1.0:
        Q = 1.0

    # Already at max quality: can't go higher
    if base_q >= max_quality_level:
        return 1.0 if target_q == base_q else 0.0

    # Stay at base quality: no upgrade at all
    if target_q == base_q:
        return 1.0 - Q

    # Number of upgrade steps
    steps = target_q - base_q

    # Intermediate tiers: base_q < target_q < max_quality_level
    if target_q < max_quality_level:
        # P = Q * α^(steps-1) * (1-α)
        return Q * (alpha ** (steps - 1)) * (1.0 - alpha)

    # Top tier: target_q == max_quality_level
    # P = Q * α^(steps-1)
    if target_q == max_quality_level:
        return Q * (alpha ** (steps - 1))

    return 0.0

def clamp01(x):
    if x < 0.0:
        return 0.0
    if x > 1.0:
        return 1.0
    return x

for item_key in all_item_variables:
    itype, iname, iq = item_key

    for recipe_key, k in products_to_recipes.get(item_key, []):
        rname, q, m, mods = recipe_key
        r = space_age["recipe"][rname]
        res = r.get("results", [])[k]

        # base amount from amount / amount_min / amount_max
        if "amount" in res:
            min_amt = res["amount"]
            max_amt = res["amount"]
        else:
            min_amt = res.get("amount_min", 0)
            max_amt = res.get("amount_max", min_amt)
            if max_amt < min_amt:
                max_amt = min_amt
        base = 0.5 * (min_amt + max_amt)

        prob = res.get("probability", 1.0)
        extra = res.get("extra_count_fraction", 0.0)
        spoiled = res.get("percent_spoiled", 0.0)  # unused, same as your code

        effects = get_effect(mods)
        Q = clamp01(effects.get("quality", 0.0))

        prod_bonus = effects.get("productivity", 0.0)
        if not r.get("allow_productivity", False):
            prod_bonus = 0.0
        else:
            max_prod = r.get("maximum_productivity", 3.0)
            if prod_bonus < 0.0:
                prod_bonus = 0.0
            if prod_bonus > max_prod:
                prod_bonus = max_prod

        expected_base = (prob * base + extra) * (1.0 + prod_bonus)
        if expected_base == 0.0:
            continue

        if res["type"] == "fluid":
            # fluids ignore quality: all fluid is quality 0
            if iq != 0:
                continue
            expected = expected_base
        else:
            # items: quality distribution
            expected = expected_base * quality_prob(q, iq, Q)

        if not expected:
            continue

        # item produced per craft of recipe_key
        G.add_edge(
            recipe_key,
            item_key,
            kind="product",
            per_machine=expected,
        )


## Stage 1: Macro model

The first stage computes how may machines are required to build the required item flow.

### Instance

- The graph `G` computed from the raw data in stage 0
- A fixed set `const` of input and output item flow required (positive if provided, negative if required)

## Variables

- `total_machines` $: \mathtt{Machines} \to \mathbb{Z}_+$ number of machines with fixed recipe, assembly machine, and module set;
- `machine_usage` $: \mathtt{Machines} \to \mathbb{R}_+$ relaxation of the `total_machines` variable, to get the precise usage of the machines;
- `used` $: \mathtt{Items} \to \mathbb{R}_+$ Total amount of item used in the factory
- `produced` $: \mathtt{Items} \to \mathbb{R}_+$ Total amount of items produced in the factory.

## Constraints

Flow constraints:

$$ \mathtt{used} \leq \mathtt{produced} $$

Relaxation of the machine usage, where $\alpha$ is the speed coeficient, computed from the assembly machine, the craft base speed and the module's effect. 

$$ \mathtt{machine\_usage} \leq \mathtt{total\_machines} \times \alpha $$


Factory utilization:

$$ \mathtt{used}[i] = \sum_{(i_1, r) \in E | i_1 = i} \mathtt{per\_machine}[(i, r)] \times \mathtt{machine\_usage}[r] $$
$$ \mathtt{produced}[i] = const[i] + \sum_{(r, i_1) \in E | i_1 = i} \mathtt{per\_machine}[(r, i)] \times \mathtt{machine\_usage}[r] $$


## Solver Parameters

In practice, a large gap of around 10% seems to be sufficent. We do not aim for optimality here. Using time limit is possible.


In [6]:
import gurobipy as grb

model = grb.Model()

A = [n for n, d in G.nodes(data=True) if d.get("kind") == "item"]
B = [n for n, d in G.nodes(data=True) if d.get("kind") == "recipe"]

total_machines = model.addVars(B, obj=100, vtype=GRB.INTEGER, name="machines")
machine_usage = model.addVars(B, name="usage")          # crafts per second
used = model.addVars(A, name="used")
produced = model.addVars(A, obj=1, name="produced")

# --- flow feasibility ---

model.addConstrs(used[i] <= produced[i] for i in A)

# --- capacity constraints (independent of per_machine) ---

for machine_key in B:
    rname, q, m, mods = machine_key
    machine = space_age[m[0]][m[1]]
    recipe = space_age["recipe"][rname]

    base_speed = machine.get("crafting_speed", 1.0)
    craft_time = recipe.get("energy_required", 0.5)

    effects = get_effect(mods)
    speed_mult = effects.get("speed", 1.0)

    eff_speed = base_speed * speed_mult
    crafts_per_second = eff_speed / craft_time

    model.addConstr(
        machine_usage[machine_key]
        <= total_machines[machine_key] * crafts_per_second
    )

# --- material balance using per_machine from the graph ---

from collections import defaultdict

expr_ingr = dict()
expr_resu = dict()

model.update()

for i in A:
    expr_u = grb.LinExpr()
    expr_p = grb.LinExpr()

    # item -> recipe edges (ingredients)
    for _, r, data in G.out_edges(i, data=True):
        if data.get("kind") != "ingredient":
            continue
        a = data["per_machine"]
        expr_ingr[(i, r)] = a * machine_usage[r]
        expr_u.addTerms(a, machine_usage[r])

    # recipe -> item edges (products)
    for r, _, data in G.in_edges(i, data=True):
        if data.get("kind") != "product":
            continue
        a = data["per_machine"]
        expr_resu[(r, i)] = a * machine_usage[r]
        expr_p.addTerms(a, machine_usage[r])

    # external sources/sinks (same as your original logic)
    if i == ("item", "scrap", 0):
        expr_p.addConstant(10000)
        produced[i].Obj = 0
    
    if i == ("fluid", "heavy-oil", 0):
        expr_p.addConstant(10000)
        produced[i].Obj = 0


    if i == ("item", "quality-module-3", 0):
        expr_p.addConstant(-0.1)


    model.addConstr(used[i] == expr_u, name=f"used-{i}")
    model.addConstr(produced[i] == expr_p, name=f"produced-{i}")

model.setParam(GRB.Param.MIPGap, 0.1)
model.setParam(GRB.Param.TimeLimit, 10)
model.optimize()


Set parameter Username
Set parameter LicenseID to value 2732003
Academic license - for non-commercial use only - expires 2026-11-03
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 10
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  10
MIPGap  0.1

Optimize a model with 52382 rows, 97192 columns and 337957 nonzeros
Model fingerprint: 0xb346a19c
Variable types: 50489 continuous, 46703 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+04]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Presolve removed 12308 rows and 20206 columns
Presolve time: 0.27s
Presolved: 40074 rows, 76986 columns, 281882 nonzeros
Variable types: 38499 continuous, 38487 integer (1 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Sh

In [7]:
s = []
for i in A:
    if used[i].X > 0:
        s += [f"{(i, used[i], produced[i].X)}"]


for m in B:
    if total_machines[m].X > 0:
        print(m, total_machines[m].X)


('holmium-plate', 0, ('assembling-machine', 'assembling-machine-3'), ('speed-module-3', 'speed-module-3')) 1.0
('superconductor', 0, ('assembling-machine', 'electromagnetic-plant'), ('speed-module-3', 'quality-module-3')) 1.0
('quality-module-2', 0, ('assembling-machine', 'electromagnetic-plant'), ('speed-module-3', 'speed-module-3', 'speed-module-3', 'speed-module-3')) 2.0
('ice-melting', 0, ('assembling-machine', 'chemical-plant'), ('speed-module-3',)) 0.9999990107084631
('holmium-solution', 0, ('assembling-machine', 'chemical-plant'), ('quality-module-3',)) 1.0
('quality-module', 0, ('assembling-machine', 'electromagnetic-plant'), ('speed-module-3', 'speed-module-3', 'speed-module-3', 'speed-module-3')) 4.0
('advanced-circuit', 0, ('assembling-machine', 'electromagnetic-plant'), ('speed-module-3', 'speed-module-3', 'speed-module-3', 'speed-module-3')) 1.0
('heavy-oil-cracking', 0, ('assembling-machine', 'chemical-plant'), ('quality-module-3', 'quality-module-3')) 1.0
('low-density-s

## Stage 2: Micro-Model

The goal of the second step is to take the generated solution of the first step and refine it further. It instanciate binary variables for each machines, and instanciate precise edges between these machines in order to satify the production and usage constraints.

Given a solution of stage 1, a solution of stage 2 is always possible, with greedy algorithm implementable. However, better solutions can be found.

In [8]:
from collections import defaultdict

# Sets of nodes from G
A = [n for n, d in G.nodes(data=True) if d.get("kind") == "item"]
B = [n for n, d in G.nodes(data=True) if d.get("kind") == "recipe"]

# Macro solution
n_r = {r: int(round(total_machines[r].X)*1.4) for r in B}
u_r = {r: machine_usage[r].X for r in B}  # crafts per second (total for recipe r)

# --- Build machine instances ---

machine_instances = []          # list of (r, k)
machine_of_inst = {}            # (r,k) -> r

for r in B:
    n = n_r[r]
    if n <= 0:
        continue
    for k in range(n):
        inst = (r, k)
        machine_instances.append(inst)
        machine_of_inst[inst] = r

# Per-instance production/consumption of each item
prod_cap = dict()   # (inst, item) -> supply rate (units/s)
cons_req = dict()   # (inst, item) -> demand rate (units/s)

producers = defaultdict(list)   # item -> [inst]
consumers = defaultdict(list)   # item -> [inst]

for inst in machine_instances:
    r = machine_of_inst[inst]
    n = n_r[r]
    if n == 0 or u_r[r] == 0.0:
        continue

    u_inst = u_r[r] / n  # crafts/s on this instance (equal split)

    # Products: r -> i
    for _, i, data in G.out_edges(r, data=True):
        if data.get("kind") != "product":
            continue
        a = data["per_machine"]       # units per craft
        cap = a * u_inst              # units/s produced by this instance
        if cap > 0.0:
            prod_cap[(inst, i)] = cap
            producers[i].append(inst)

    # Ingredients: i -> r
    for i, _, data in G.in_edges(r, data=True):
        if data.get("kind") != "ingredient":
            continue
        a = data["per_machine"]       # units per craft
        req = a * u_inst              # units/s consumed by this instance
        if req > 0.0:
            cons_req[(inst, i)] = req
            consumers[i].append(inst)

# Items that are actually routed between machines
items_routed = [i for i in A if producers[i] and consumers[i]]

In [9]:
micro = grb.Model("micro_routing")

flows = {}   # (i, p, c) -> f variable
links = {}   # (i, p, c) -> y variable

# Big-M per item: total production of that item (upper bound on any single link)
M_item = {}
for i in items_routed:
    M_item[i] = sum(prod_cap[(p, i)] for p in producers[i])

# Create variables
for i in items_routed:
    for p in producers[i]:
        for c in consumers[i]:
            f = micro.addVar(lb=0.0, name="flow")
            y = micro.addVar(vtype=grb.GRB.BINARY)
            flows[(i, p, c)] = f
            links[(i, p, c)] = y

micro.update()

# --- Constraints ---

# Producer capacity: sum_c f[i, p, c] ≤ prod_cap[p, i]
for (p, i), cap in prod_cap.items():
    if i not in items_routed:
        continue
    relevant_cs = [c for c in consumers[i] if (i, p, c) in flows]
    if not relevant_cs:
        continue
    micro.addConstr(
        grb.quicksum(flows[(i, p, c)] for c in relevant_cs) <= cap,
        name=f"prod_cap_{p}_{i}",
    )

# Consumer demand: sum_p f[i, p, c] ≥ cons_req[c, i]
for (c, i), req in cons_req.items():
    if i not in items_routed:
        continue
    relevant_ps = [p for p in producers[i] if (i, p, c) in flows]
    if not relevant_ps:
        continue
    micro.addConstr(
        grb.quicksum(flows[(i, p, c)] for p in relevant_ps) >= req,
        name=f"cons_req_{c}_{i}",
    )

# Link activation: f[i, p, c] ≤ M_i * y[i, p, c]
for (i, p, c), f in flows.items():
    micro.addConstr(
        f <= M_item[i] * links[(i, p, c)]
    )

# --- Objective: minimize number of links ---

micro.setObjective(
    grb.quicksum(links.values()),
    sense=grb.GRB.MINIMIZE,
)

micro.write("micro.lp")
micro.setParam(GRB.Param.MIPGap, 0.1)
micro.optimize()


Set parameter MIPGap to value 0.1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
MIPGap  0.1

Optimize a model with 1462 rows, 2058 columns and 4116 nonzeros
Model fingerprint: 0xfbe09711
Variable types: 1029 continuous, 1029 integer (1029 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 7e+00]
Presolve removed 612 rows and 606 columns
Presolve time: 0.13s
Presolved: 850 rows, 1452 columns, 2904 nonzeros
Variable types: 726 continuous, 726 integer (726 binary)
Found heuristic solution: objective 812.0000000
Found heuristic solution: objective 811.0000000
Found heuristic solution: objective 810.0000000

Root relaxation: objective 1.970000e+02, 1090 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current 

#### Final graph generation

The final graph is a graph between machines. Edges correspond to flow of items between the machines. This graph is exported, and can be drawn to be then plotted inside factorio as a blueprint.

In [10]:
import networkx as nx

# H: micro-level machine graph
H = nx.MultiDiGraph()

machine_to_int = dict()
# 1. Add machine-instance nodes
for inst in machine_instances:
    r = machine_of_inst[inst]   # original recipe/machine node
    (rname, q, m, mods), k = inst
    H.add_node(len(machine_to_int), recipe=rname, quality=q, machine="/".join(m), modules=";".join(mods), index=k)
    
    machine_to_int[inst] = len(machine_to_int)

# 2. Add edges for positive flows
for (item_key, prod_inst, cons_inst), fvar in flows.items():
    flow = fvar.X

    if flow <= 0.0:
        continue

    # one edge per (item, producer, consumer)
    H.add_edge(
        machine_to_int[prod_inst],
        machine_to_int[cons_inst],
        item=item_key,
        flow=flow,
    )


In [11]:
nx.write_gml(H, "data/micro.gml")